In [2]:
%%bash

git clone https://github.com/keras-team/keras-tuner.git
cd keras-tuner
pip install .

Processing /home/jovyan/notebooks/keras-tuner
  Stored in directory: /home/jovyan/.cache/pip/wheels/bc/a7/b7/1bfe6ecb245a5d7865c32ea1013bad996959f6ef3fd23e1d67
Successfully built Keras-Tuner
  Found existing installation: Keras-Tuner 0.9.0.1562626578
    Uninstalling Keras-Tuner-0.9.0.1562626578:
      Successfully uninstalled Keras-Tuner-0.9.0.1562626578


fatal: destination path 'keras-tuner' already exists and is not an empty directory.
You are using pip version 19.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Verify that we're using tensorflow 2.0

In [3]:
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters


(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
x = x.astype('float32') / 255.
val_x = val_x.astype('float32') / 255.

x = x[:10000]
y = y[:10000]

Here's how to perform hyperparameter tuning for a single-layer dense neural network using random search.

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Range('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

Case #1:  Basic
- Define a `build_model` function
- Returns a compiled model
- Use hyperparameters defined on the fly

The search space may contain conditional hyperparameters.

Below, we have a for loop creating a tunable number of layers, which themselves involve a tunable units parameter.

This can be pushed to any level of parameter interdependency, including recursion.

Note that all parameter names should be unique (here, in the loop over i, we name the inner parameters 'units_' + str(i)).

In [5]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.Range('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Range('units_' + str(i), 32, 512, 32),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband.

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [6]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='tuner_results',
    project_name='helloworld_case_1')

You can print a summary of the search space:



In [7]:
tuner.search_space_summary()

Then, start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().

Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [ ]:
tuner.search(x=x,
             y=y,
             epochs=3,
             validation_data=(val_x, val_y))

W0708 23:07:34.102058 140369864963904 deprecation.py:323] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Name,Best model,Current model
accuracy,0.943,0.943
loss,0.1952,0.1952
val_loss,0.2922,0.2922
val_accuracy,0.9237,0.9237


When search is over, you can retrieve the best model(s):

In [ ]:
models = tuner.get_best_models(num_models=2)
print(models)

Or print a summary of the results:



In [ ]:
tuner.results_summary()

You will also find detailed logs and checkpoints in the folder `results/helloworld`



In [ ]:
!ls -al tuner_results

Case #2:
- Override the loss and metrics

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    loss=keras.losses.SparseCategoricalCrossentropy(name='my_loss'),
    metrics=['accuracy', 'mse'],
    max_trials=5,
    directory='tuner_results',
    project_name='helloworld_case_2')

tuner.search(x, y,
             epochs=5,
             validation_data=(val_x, val_y))

Case #3:
- We define a custom HyperModel subclass instead of model-building function
- This makes it easy to share and reuse hypermodels.
- A HyperModel subclass only needs to implement a build(self, hp) method.

In [ ]:
class MyHyperModel(HyperModel):

    def __init__(self, img_size, num_classes):
        self.img_size = img_size
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten(input_shape=self.img_size))
        for i in range(hp.Range('num_layers', 2, 20)):
            model.add(layers.Dense(units=hp.Range('units_' + str(i), 32, 512, 32),
                                   activation='relu'))
        model.add(layers.Dense(self.num_classes, activation='softmax'))
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
        return model


In [ ]:
tuner = RandomSearch(
    MyHyperModel(img_size=(28, 28), num_classes=10),
    objective='val_accuracy',
    max_trials=5,
    directory='results',
    project_name='helloworld_case_3')

tuner.search(x,
             y=y,
             epochs=5,
             validation_data=(val_x, val_y))

Case #4:
- Restrict the search space
- Use default values for params that are left out

In [ ]:
hp = HyperParameters()
hp.Choice('learning_rate', [1e-1, 1e-3])

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=False,
    objective='val_accuracy',
    directory='tuner_results',
    project_name='helloworld_case_4')

tuner.search(x=x,
             y=y,
             epochs=5,
             validation_data=(val_x, val_y))

Case #5:
- We override specific parameters with fixed values that aren't the default

In [ ]:
hp = HyperParameters()
hp.Fixed('learning_rate', 0.1)

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=True,
    objective='val_accuracy',
    directory='results',
    project_name='helloworld_case_5')

tuner.search(x=x,
             y=y,
             epochs=5,
             validation_data=(val_x, val_y))

Case #6:
- We reparameterize the search space
- This means that we override the distribution of specific hyperparameters

In [ ]:
hp = HyperParameters()
hp.Choice('learning_rate', [1e-1, 1e-3])

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=True,
    objective='val_accuracy',
    directory='results',
    project_name='helloworld_case_6')

tuner.search(x=x,
             y=y,
             epochs=5,
             validation_data=(val_x, val_y))

Case #7:
- We predefine the search space
- No unregistered parameters are allowed in `build`

In [ ]:
hp = HyperParameters()
hp.Choice('learning_rate', [1e-1, 1e-3])
hp.Range('num_layers', 2, 20)

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.get('num_layers')):
        model.add(layers.Dense(32,
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    allow_new_entries=False,
    objective='val_accuracy',
    directory='results',
    project_name='helloworld_case_7')

tuner.search(x=x,
             y=y,
             epochs=5,
             validation_data=(val_x, val_y))

Keras Tuner includes pre-made tunable applications: HyperResNet and HyperXception.
    
These are ready-to-use hypermodels for computer vision.

They come pre-compiled with loss="categorical_crossentropy" and metrics=["accuracy"].


In [ ]:
from kerastuner.applications import HyperResnet
from kerastuner.tuners import Hyperband

hypermodel = HyperResnet(input_shape=(128, 128, 3), num_classes=10)

tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_trials=40,
    directory='my_dir',
    project_name='helloworld')

tuner.search(x, y,
             epochs=20,
             validation_data=(val_x, val_y))

You can easily restrict the search space to just a few parameters

If you have an existing hypermodel, and you want to search over only a few parameters (such as the learning rate), you can do so by passing a hyperparameters argument to the tuner constructor, as well as tune_new_entries=False to specify that parameters that you didn't list in hyperparameters should not be tuned. For these parameters, the default value gets used.



In [ ]:
from kerastuner import HyperParameters

hypermodel = HyperXception(input_shape=(128, 128, 3), num_classes=10)

hp = HyperParameters()
# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

tuner = Hyperband(
    hypermodel,
    hyperparameters=hp,
    # `tune_new_entries=False` prevents unlisted parameters from being tuned
    tune_new_entries=False,
    objective='val_accuracy',
    max_trials=40,
    directory='my_dir',
    project_name='helloworld')

tuner.search(x, y,
             epochs=20,
             validation_data=(val_x, val_y))

Want to know what parameter names are available? Read the code.

About parameter default values: 
- Whenever you register a hyperparameter inside a model-building function or the build method of a hypermodel, you can specify a default value:

In [ ]:
hp.Range('units',
         min_value=32,
         max_value=512,
         step=32,
         default=128)

If you don't, hyperparameters always have a default default (for Range, it is equal to min_value).

Fixing values in a hypermodel:
- What if you want to do the reverse -- tune all available parameters in a hypermodel, except one (the learning rate)?
- Pass a hyperparameters argument with a Fixed entry (or any number of Fixed entries), and specify tune_new_entries=True.


In [ ]:
hypermodel = HyperXception(input_shape=(128, 128, 3), num_classes=10)

hp = HyperParameters()
hp.Fixed('learning_rate', value=1e-4)

tuner = Hyperband(
    hypermodel,
    hyperparameters=hp,
    tune_new_entries=True,
    objective='val_accuracy',
    max_trials=40,
    directory='my_dir',
    project_name='helloworld')

tuner.search(x, y,
             epochs=20,
             validation_data=(val_x, val_y))

Overriding compilation arguments
- If you have a hypermodel for which you want to change the existing optimizer, loss, or metrics, you can do so by passing these arguments to the tuner constructor:

In [ ]:
hypermodel = HyperXception(input_shape=(128, 128, 3), num_classes=10)

tuner = Hyperband(
    hypermodel,
    optimizer=keras.optimizers.Adam(1e-3),
    loss='mse',
    metrics=[keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall')],
    objective='val_precision',
    max_trials=40,
    directory='my_dir',
    project_name='helloworld')

tuner.search(x, y,
             epochs=20,
             validation_data=(val_x, val_y))